<a href="https://colab.research.google.com/github/dawoodshahzad07/Transfer_Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Weight Initializers:
Zero: Not recommended.
Random: Basic, breaks symmetry.
Xavier/Glorot: Good for tanh, sigmoid.
He: Best for ReLU variants.
LeCun: Useful for tanh, sigmoid.
```
---
```
Activation Functions:
Sigmoid: Binary classification, vanishing gradient.
Tanh: Zero-centered, vanishing gradient.
ReLU: General use, efficient, avoids vanishing gradient.
Leaky ReLU: Mitigates dying ReLU problem.
ELU: Useful for deeper networks.
Softmax: Output layer for multi-class classification.
```

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 266610 

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 1.5144 - accuracy: 0.6016 - val_loss: 0.9318 - val_accuracy: 0.8024 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.7459 - accuracy: 0.8239 - val_loss: 0.5862 - val_accuracy: 0.8552 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 6s - loss: 0.5443 - accuracy: 0.8599 - val_loss: 0.4696 - val_accuracy: 0.8784 - 6s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 7s - loss: 0.4619 - accuracy: 0.8758 - val_loss: 0.4124 - val_accuracy: 0.8906 - 7s/epoch - 4ms/step
Epoch 5/10
1719/1719 - 6s - loss: 0.4168 - accuracy: 0.8854 - val_loss: 0.3779 - val_accuracy: 0.8986 - 6s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 6s - loss: 0.3877 - accuracy: 0.8915 - val_loss: 0.3547 - val_accuracy: 0.9036 - 6s/epoch - 4ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.3669 - accuracy: 0.8964 - val_loss: 0.3375 - val_accuracy: 0.9060 - 4s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.3509 - accuracy: 0.9004 - val_loss: 0.3238 - val_accuracy: 0.9112 - 5s/epoch - 3ms/step
Epoch 9/

In [ ]:
model.save("full_mnist_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
load_full_mnist_model = tf.keras.models.load_model("full_mnist_model.h5")

In [ ]:
load_full_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 266610 

#Transfer Learning:

In [ ]:
for layer in load_full_mnist_model.layers:
  print(f"{layer.name} : {layer.trainable}")

flatten : True
dense : True
leaky_re_lu : True
dense_1 : True
leaky_re_lu_1 : True
dense_2 : True


In [ ]:
for layer in load_full_mnist_model.layers[:-1]:
  print(f"{layer.name} : {layer.trainable}")

flatten : True
dense : True
leaky_re_lu : True
dense_1 : True
leaky_re_lu_1 : True


In [ ]:
for layer in load_full_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False


In [ ]:
for layer in load_full_mnist_model.layers:
  print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False
dense_2 : True


In [ ]:
lower_pretrained_layers = load_full_mnist_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265802 (1.01 MB)
Trainable params: 202 (

In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
 def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels


In [ ]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [ ]:
np.unique(y_train_bin)

array([0, 1])

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                  metrics=["accuracy"])

In [ ]:
history = new_model.fit(X_train, y_train_bin, epochs= 10, validation_data= (X_valid, y_valid_bin), verbose=1)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4495 - accuracy: 0.7961 - val_loss: 0.3256 - val_accuracy: 0.8702
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3299 - accuracy: 0.8637 - val_loss: 0.3016 - val_accuracy: 0.8790
Epoch 3/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3158 - accuracy: 0.8690 - val_loss: 0.2914 - val_accuracy: 0.8818
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3089 - accuracy: 0.8722 - val_loss: 0.2859 - val_accuracy: 0.8842
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3042 - accuracy: 0.8741 - val_loss: 0.2817 - val_accuracy: 0.8870
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3009 - accuracy: 0.8757 - val_loss: 0.2784 - val_accuracy: 0.8856
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2980 - accuracy: 0.8769 - val_loss: 0.2759 - val_accuracy:

In [ ]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2836 - accuracy: 0.8861


[0.28360113501548767, 0.8860999941825867]

In [ ]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [ ]:
np.argmax(new_model.predict(X_new), axis=1)

1/1 [==============================] - 0s 81ms/step


array([0, 1, 0])

<h1> Task </h1> <br>
if label > 5 == 1 <br>
if label <= 5 == 0

In [ ]:
new_model.save("task.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
load_task_model = tf.keras.models.load_model("task.h5")

In [ ]:
load_task_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265802 (1.01 MB)
Trainable params: 202 (

In [ ]:
for layer in load_task_model.layers:
  print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False
dense_3 : True


In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
def update_low_high_labels(labels):
  for idx, label in enumerate(labels):
    print(f"{idx} : {label}")
  return labels
update_low_high_labels([y_train, y_test, y_valid])

0 : [7 3 4 ... 5 6 8]
1 : [7 2 1 ... 4 5 6]
2 : [5 0 4 ... 2 1 2]


[array([7, 3, 4, ..., 5, 6, 8], dtype=uint8),
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8),
 array([5, 0, 4, ..., 2, 1, 2], dtype=uint8)]

In [ ]:
def update_low_high_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label > 5, 1, 0)
  return labels

In [ ]:
update_low_high_labels([y_train, y_test, y_valid])

[array([1, 0, 0, ..., 0, 1, 1]),
 array([1, 0, 0, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 0])]

In [ ]:
y_train_bin, y_test_bin, y_valid_bin = update_low_high_labels([y_train, y_test, y_valid])

In [ ]:
np.unique(y_train_bin)

array([0, 1])

In [ ]:
load_task_model.compile(loss = "sparse_categorical_crossentropy",
                        optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
metrics=["accuracy"])

In [ ]:
history_task = load_task_model.fit(X_train, y_train_bin, epochs=10, validation_data=(X_valid, y_valid_bin),verbose=1)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6204 - accuracy: 0.6953 - val_loss: 0.4659 - val_accuracy: 0.7684
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4288 - accuracy: 0.8033 - val_loss: 0.3971 - val_accuracy: 0.8276
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3910 - accuracy: 0.8337 - val_loss: 0.3704 - val_accuracy: 0.8456
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3738 - accuracy: 0.8440 - val_loss: 0.3567 - val_accuracy: 0.8506
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3636 - accuracy: 0.8489 - val_loss: 0.3469 - val_accuracy: 0.8556
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3565 - accuracy: 0.8517 - val_loss: 0.3408 - val_accuracy: 0.8584
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3513 - accuracy: 0.8539 - val_loss: 0.3356 - val_accuracy:

In [ ]:
load_task_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.3357 - accuracy: 0.8630


[0.3357034921646118, 0.8629999756813049]

In [ ]:
y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([1, 0, 0]))

In [ ]:
X_new = X_test[:3]

In [ ]:
np.argmax(load_task_model.predict(X_new), axis=1)

1/1 [==============================] - 0s 24ms/step


array([1, 0, 0])